In [12]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ------------------------------------------------------
# 1️⃣ 데이터 로드 및 전처리 (원본 데이터셋: microbiology_qna.csv)
# ------------------------------------------------------
df = pd.read_csv("microbiology_qna.csv")

# 📌 결측값 제거
df.dropna(inplace=True)

# 📌 텍스트 정제 함수
def clean_text(text):
    text = text.lower()  # 소문자 변환
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # 특수문자 제거
    return text

# 📌 질문 데이터 정제
df["cleaned_question"] = df["question"].apply(clean_text)

# ------------------------------------------------------
# 2️⃣ TF-IDF 벡터화
# ------------------------------------------------------
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df["cleaned_question"])

# TF-IDF 벡터가 학습한 단어 리스트 확인 (디버깅용)
feature_names = vectorizer.get_feature_names_out()
print(f"TF-IDF가 학습한 단어 개수: {len(feature_names)}")
print(f"학습된 단어 샘플: {feature_names[:20]}")  # 앞에서 20개만 확인

# ------------------------------------------------------
# 3️⃣ 챗봇 응답 함수 (TF-IDF + 코사인 유사도)
# ------------------------------------------------------
def chatbot_response(user_input):
    """
    TF-IDF 코사인 유사도를 기반으로
    가장 유사한 질문을 찾아 답변을 반환
    """
    # 입력된 질문 전처리 및 벡터화
    cleaned_input = clean_text(user_input)
    user_vector = vectorizer.transform([cleaned_input])
    
    # 디버깅: 입력 벡터 확인
    input_vector_array = user_vector.toarray()
    print(f"\n📌 입력된 질문 벡터 (일부 출력): {input_vector_array[:, :20]}")  # 앞 20개 요소만 확인
    
    # 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_vector, tfidf_matrix).flatten()
    
    # 가장 유사한 질문 인덱스 찾기
    best_match_idx = np.argmax(similarity_scores)
    best_score = similarity_scores[best_match_idx]

    # 유사도 점수가 너무 낮으면 기본 응답 반환
    if best_score < 0.1:
        return "No relevant answer found", "I'm sorry, but I couldn't find a relevant answer.", best_score

    # 가장 유사한 질문과 답변 반환
    best_question = df.iloc[best_match_idx]["question"]
    best_answer = df.iloc[best_match_idx]["answer"]
    
    return best_question, best_answer, best_score

# ------------------------------------------------------
# 4️⃣ 가상의 유사 질문 데이터 로드 (Generated_Question_Pairs.csv)
# ------------------------------------------------------
# CSV 파일에는 'original_question', 'similar_question' 컬럼이 존재해야 합니다.
test_pairs_df = pd.read_csv("Generated_Question_Pairs.csv")

# 📌 결측값 제거 (원본질문/유사질문 둘 다)
test_pairs_df.dropna(subset=["original_question", "similar_question"], inplace=True)

# ------------------------------------------------------
# 5️⃣ 챗봇 정확도 평가 (유사도 기반)
# ------------------------------------------------------
correct = 0
total = len(test_pairs_df)
threshold = 0.8  # 유사도가 0.8 이상이면 정답으로 인정

for idx, row in test_pairs_df.iterrows():
    generated_question = row["similar_question"]
    original_question = row["original_question"]
    
    # 챗봇 응답
    final_question, _, score = chatbot_response(generated_question)
    
    # 유사도 점수가 threshold 이상이면 정답으로 간주
    if score >= threshold:
        correct += 1

# ------------------------------------------------------
# 6️⃣ 정확도 계산 및 결과 출력
# ------------------------------------------------------
accuracy = (correct / total) * 100
print(f"\n✅ 유사도 기반 챗봇 정확도: {accuracy:.2f}% (기준 유사도: {threshold})")

TF-IDF가 학습한 단어 개수: 8783
학습된 단어 샘플: ['02' '050' '10' '100' '1000' '10000' '10000year' '10002000' '100watt'
 '101' '1020' '1030c' '10m' '10mw' '11' '114b' '12' '129i' '129i127i'
 '12c']

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

📌 입력된 질문 벡터 (일부 출력): [[0. 0. 0. 0. 0. 0. 

In [13]:
# ------------------------------------------------------
# 7️⃣ 사용자 입력을 받아 챗봇 실행 (실제 챗봇 대화)
# ------------------------------------------------------
while True:
    user_input = input("\n💬 질문을 입력하세요 (종료: 'exit'): ").strip()
    if user_input.lower() == "exit":
        print("🔚 챗봇을 종료합니다.")
        break
    
    best_question, best_answer, best_score = chatbot_response(user_input)
    
    print(f"\n🤖 가장 유사한 질문: {best_question}")
    print(f"✅ 답변: {best_answer}")
    print(f"📊 유사도 점수: {best_score:.2f}")

🔚 챗봇을 종료합니다.


In [14]:
import pickle

# 📌 모델 및 벡터 저장
with open("tfidf_chatbot_model.pkl", "wb") as f:
    pickle.dump({"vectorizer": vectorizer, "tfidf_matrix": tfidf_matrix, "df": df}, f)

print("✅ TF-IDF 모델이 'tfidf_chatbot_model.pkl' 파일로 저장되었습니다.")

✅ TF-IDF 모델이 'tfidf_chatbot_model.pkl' 파일로 저장되었습니다.


In [ ]:
# 📌 저장된 모델 불러오기
with open("tfidf_chatbot_model.pkl", "rb") as f:
    loaded_data = pickle.load(f)

vectorizer = loaded_data["vectorizer"]
tfidf_matrix = loaded_data["tfidf_matrix"]
df = loaded_data["df"]

print("✅ 저장된 TF-IDF 모델을 로드했습니다.")